#Credit
[Parler TTS HuggingFace Demo](https://huggingface.co/spaces/parler-tts/parler_tts_mini)

[Parler TTS Github](https://github.com/huggingface/parler-tts)

##Modification


*   Support longer text
*   Remove silence from TTS



In [ ]:
#@title Install Parler TTS and Restart Session
!pip install git+https://github.com/ylacombe/parler-tts.git@parler-tts-release
!pip install gradio
!pip install --upgrade protobuf
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!import soundfile as sf
!pip install auto-editor
from IPython.display import clear_output
clear_output()
from google.colab import output
output.eval_js('new Audio("https://github.com/neuralfalcon/Roop-Image-FaceSwap/raw/main/install_voice.mp3").play()')
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [ ]:
#@title Download Model
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import gradio as gr
import torch
import soundfile as sf
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, AutoFeatureExtractor, set_seed
import nltk
nltk.download('punkt')
device = "cuda:0" if torch.cuda.is_available() else "cpu"


repo_id = "parler-tts/parler_tts_mini_v0.1"

model = ParlerTTSForConditionalGeneration.from_pretrained(repo_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(repo_id)
feature_extractor = AutoFeatureExtractor.from_pretrained(repo_id)


SAMPLE_RATE = feature_extractor.sampling_rate
from IPython.display import clear_output
clear_output()


In [148]:
#@title Utils
# SEED = 42
SEED = 4294967295
if not os.path.exists('./results'):
  os.mkdir('./results')
def gen_tts(text, description):
    global SEED
    inputs = tokenizer(description, return_tensors="pt").to(device)
    prompt = tokenizer(text, return_tensors="pt").to(device)

    set_seed(SEED)
    generation = model.generate(
        input_ids=inputs.input_ids, prompt_input_ids=prompt.input_ids, do_sample=True, temperature=1.0
    )
    audio_arr = generation.cpu().numpy().squeeze()

    return SAMPLE_RATE, audio_arr

import os
import shutil
from pydub import AudioSegment
import uuid
import subprocess

def get_temp_name(file_name):
  ori_name=os.path.basename(file_name)
  file_name, file_extension = os.path.splitext(ori_name)
  random_uuid = str(uuid.uuid4())[:6]
  new_name=f'{file_name}_{random_uuid}{file_extension}'
  return new_name
def remove_silence_from_audio(audio_file, margin):
    base_path = os.path.dirname(audio_file)
    random_uuid = str(uuid.uuid4())[:6]
    new_save_path = None
    logs=None
    try:
        ffprobe_command = f"ffprobe -i {audio_file} -show_entries format=duration -v quiet -of csv=p=0"
        duration_string = subprocess.check_output(ffprobe_command, shell=True, text=True)
        duration = float(duration_string)
        # print(f"Audio duration {duration} seconds")

        ffmpeg_command = f"ffmpeg -f lavfi -t {duration} -i color=c=black:s=160x90 -c:v libx264 -strict experimental {base_path}/blank.mp4 -y"

        white_var = os.system(ffmpeg_command)

        if white_var == 0:
            response_var = os.system(f"ffmpeg -i {base_path}/blank.mp4 -i {audio_file} -map 0 -map 1 -c:v copy -c:a aac -strict experimental {base_path}/temp.mp4 -y")

            if response_var == 0:
                var = os.system(f"auto-editor {base_path}/temp.mp4 --margin {float(margin)}sec")

                if var == 0:
                    # new_save_path = f"{base_path}/{random_uuid}.wav"
                    new_save_path = f"./results/remove_silence_{get_temp_name(audio_file)}"

                    final_var = os.system(f"ffmpeg -i {base_path}/temp_ALTERED.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 {new_save_path} -y")
                    if final_var == 0:
                        ffprobe_command = f"ffprobe -i {new_save_path} -show_entries format=duration -v quiet -of csv=p=0"
                        after_duration_string = subprocess.check_output(ffprobe_command, shell=True, text=True)
                        after_duration = float(after_duration_string)
                        logs=f"""
                        Before: {duration} sec
                        \n
                        After: {after_duration} sec
                        """
                        print(f"Audio saved")

                    else:
                        print("Failed to remove silence from audio")
                else:
                    print("Failed to edit video")
            else:
                print("Can't add audio to blank video")
        else:
            print("Can't create blank video")

    except Exception as e:
        print(e)
        print("Can't find audio duration")
    return new_save_path,logs
def merge_audio_files(temp_list, output_path):
    # Initialize an empty AudioSegment object to store the merged audio
    merged_audio = AudioSegment.empty()

    # Iterate over each audio file path in temp_list
    for audio_file in temp_list:
        # Load the audio file using Pydub
        audio = AudioSegment.from_wav(audio_file)

        # Append the loaded audio to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output_path
    merged_audio.export(output_path, format="wav")
import uuid

def name_from_text(text):
    # Generate a random UUID hex string
    random_uuid = uuid.uuid4().hex[:6]
    if len(text)==0:
        raise ValueError("Text must be at least 20 characters long.")
        filename = f"{random_uuid}.wav"
    else:
      # Extract the first 20 characters from the text
      truncated_text = text[:20]
      # Replace special characters with underscores
      sanitized_text = truncated_text.replace("'", "_").replace(".", "_").replace('"', "_")
      # Concatenate truncated text and UUID to form the filename
      filename = f"{sanitized_text}_{random_uuid}.wav"

    return './results/'+filename.replace(" ","_")

def voice_design(text,description,large=False,remove_silence_from_tts=False,silence_margin='0.2'):
  save_path=name_from_text(text)
  # random_uuid = uuid.uuid4().hex[:6]
  # save_path = f"./results/{random_uuid}.wav"
  if os.path.exists("./chunks_audio"):
    shutil.rmtree("./chunks_audio")
  os.mkdir("./chunks_audio")
  if large:
    sentences = nltk.sent_tokenize(text)
    temp_list=[]
    for i,sentence in enumerate(sentences):
      # print(f'{i} : {sentence}')
      try:
        temp_path=f"./chunks_audio/{i}.wav"
        SAMPLE_RATE,audio_arr=gen_tts(sentence, description)
        sf.write(temp_path, audio_arr, SAMPLE_RATE)
        temp_list.append(temp_path)
      except:
        pass
    if len(temp_list)==1:
      shutil.copy(temp_list[0],save_path)
    elif len(temp_list)>1:
      merge_audio_files(temp_list,save_path)
    else:
      print("No audio")


  else:
    SAMPLE_RATE,audio_arr=gen_tts(text, description)
    sf.write(save_path, audio_arr, SAMPLE_RATE)
  if remove_silence_from_tts:
    remove_silence_save_path,log_data=remove_silence_from_audio(save_path, silence_margin)
    return remove_silence_save_path,remove_silence_save_path,log_data
  return save_path,save_path,''




In [147]:
#@title Colab Demo
text="Why don't skeletons fight each other?  Because they don't have the guts!" # @param {type: "string"}
description="A female speaker with a slightly low-pitched, quite monotone voice delivers her words at a slightly faster-than-average pace in a confined space with very clear audio." # @param {type: "string"}
# save_path="/content/demo.wav" # @param {type: "string"}

long_text = False #@param {type:"boolean"}
remove_silence_from_tts= False #@param {type:"boolean"}
silence_margin="0.1" # @param {type: "string"}
save_path,_,logs_data=voice_design(text,description,long_text,remove_silence_from_tts,silence_margin)
from IPython.display import clear_output
clear_output()

print(f"TTS file save at : {os.getcwd()}{save_path[1:]}")
import IPython.display as ipd
ipd.Audio(save_path,autoplay=True)

TTS file save at : /content/results/Why_don_t_skeletons__4bea5f.wav


In [ ]:
#@title Run Gradio Webapp
default_text = "Please surprise me and speak in whatever voice you enjoy."
examples = [
    [
        "Remember - this is only the first iteration of the model! To improve the prosody and naturalness of the speech further, we're scaling up the amount of training data by a factor of five times.",
        "A male speaker with a low-pitched voice delivering his words at a fast pace in a small, confined space with a very clear audio and an animated tone.",False,False,0.0
    ],
    [
        "'This is the best time of my life, Bartley,' she said happily.",
        "A female speaker with a slightly low-pitched, quite monotone voice delivers her words at a slightly faster-than-average pace in a confined space with very clear audio.",False,False,0.0
    ],
    [
        "In a quiet village bordered by woods, lived Elara, a young girl captivated by the mysteries of nature. One day, while exploring the forest, she stumbled upon an ancient tree with a hidden treasure—a shimmering amulet. As she held the amulet, she felt a surge of magical energy course through her veins, connecting her to the essence of the woods. From that moment on, Elara became the Guardian of the Forest, using her newfound powers to protect its inhabitants. With the amulet as her guide, she embarked on countless adventures, her heart forever intertwined with the magic of the woods.",
        "A male speaker with a slightly high-pitched voice delivering his words at a slightly slow pace in a small, confined space with a touch of background noise and a quite monotone tone.",True,True,0.1
    ],
    [
        "Montrose also, after having experienced still more variety of good and bad fortune, threw down his arms, and retired out of the kingdom.",
        "A male speaker with a low-pitched voice delivers his words at a fast pace and an animated tone, in a very spacious environment, accompanied by noticeable background noise.",False,True,0.1
    ],
]
css = """
        #share-btn-container {
            display: flex;
            padding-left: 0.5rem !important;
            padding-right: 0.5rem !important;
            background-color: #000000;
            justify-content: center;
            align-items: center;
            border-radius: 9999px !important;
            width: 13rem;
            margin-top: 10px;
            margin-left: auto;
            flex: unset !important;
        }
        #share-btn {
            all: initial;
            color: #ffffff;
            font-weight: 600;
            cursor: pointer;
            font-family: 'IBM Plex Sans', sans-serif;
            margin-left: 0.5rem !important;
            padding-top: 0.25rem !important;
            padding-bottom: 0.25rem !important;
            right:0;
        }
        #share-btn * {
            all: unset !important;
        }
        #share-btn-container div:nth-child(-n+2){
            width: auto !important;
            min-height: 0px !important;
        }
        #share-btn-container .wrap {
            display: none !important;
        }
"""
with gr.Blocks(css=css) as block:
    gr.HTML(
        """
            <div style="text-align: center; max-width: 700px; margin: 0 auto;">
              <div
                style="
                  display: inline-flex; align-items: center; gap: 0.8rem; font-size: 1.75rem;
                "
              >
                <h1 style="font-weight: 900; margin-bottom: 7px; line-height: normal;">
                  Parler-TTS 🗣️
                </h1>
              </div>
            </div>
        """
    )
    gr.HTML(
        f"""
        <p><a href="https://github.com/huggingface/parler-tts"> Parler-TTS</a> is a training and inference library for
        high-fidelity text-to-speech (TTS) models. The model demonstrated here, <a href="https://huggingface.co/parler-tts/parler_tts_mini_v0.1"> Parler-TTS Mini v0.1</a>,
        is the first iteration model trained using 10k hours of narrated audiobooks. It generates high-quality speech
        with features that can be controlled using a simple text prompt (e.g. gender, background noise, speaking rate, pitch and reverberation).</p>

        <p>Tips for ensuring good generation:
        <ul>
            <li>Include the term "very clear audio" to generate the highest quality audio, and "very noisy audio" for high levels of background noise</li>
            <li>Punctuation can be used to control the prosody of the generations, e.g. use commas to add small breaks in speech</li>
            <li>The remaining speech features (gender, speaking rate, pitch and reverberation) can be controlled directly through the prompt</li>
        </ul>
        </p>
        """
    )
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="Input Text", lines=2, value=default_text, elem_id="input_text")
            description = gr.Textbox(label="Description", lines=2, value="", elem_id="input_description")
            long_text=gr.Checkbox(label="If your input text is long",value=False)
            remove_silence_from_tts=gr.Checkbox(label="Remove Silence From TTS",value=False)
            silence_margin=gr.Slider(0, 1, value=0.1, label="Add 0.2 seconds of padding before and after to make the edit nicer", info="Choose between 0 and 10")
            run_button = gr.Button("Generate Audio", variant="primary")
        with gr.Column():
            audio_out = gr.Audio(label="Parler-TTS generation", type="filepath", elem_id="audio_out")
            raw_file=gr.File(label="Download Raw Audio File")
            silence_logs = gr.Textbox(label="Logs", elem_id="Silence log")

    inputs = [input_text, description,long_text,remove_silence_from_tts,silence_margin]
    outputs = [audio_out,raw_file,silence_logs]
    gr.Examples(examples=examples, fn=voice_design, inputs=inputs, outputs=outputs, cache_examples=False)
    run_button.click(fn=voice_design, inputs=inputs, outputs=outputs, queue=True)
    gr.HTML(
        """
        <p>To improve the prosody and naturalness of the speech further, we're scaling up the amount of training data to 50k hours of speech.
        The v1 release of the model will be trained on this data, as well as inference optimisations, such as flash attention
        and torch compile, that will improve the latency by 2-4x. If you want to find out more about how this model was trained and even fine-tune it yourself, check-out the
        <a href="https://github.com/huggingface/parler-tts"> Parler-TTS</a> repository on GitHub.</p>

        <p>The Parler-TTS codebase and its associated checkpoints are licensed under <a href='https://github.com/huggingface/parler-tts?tab=Apache-2.0-1-ov-file#readme'> Apache 2.0</a>.</p>
        """
    )

block.queue()
block.launch(share=True,debug=False)